In [1]:
from core.libs import pd
from core.db import get_engine
from core.schema_helpers import rename_columns_using_schema
from Cruises.general_importer import prepare_df_for_sql, save_inventory_to_sql, ensure_table
from Cruises.dead_alive_calculator import calculate_dead_alive
from core.doyle_calculator import calculate_doyle

archivo = r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\USA\2021_ForestInventory\9-Database\2021_US_Database v1.1.xlsx"
sheet = 'TreeList_Data'

In [2]:
#Leer el archivo
df_original = pd.read_excel(archivo, sheet_name=sheet)
print("📄 Columnas originales:", list(df_original.columns))

df = df_original.drop(columns=["DAP (cm)", "THt (m)", "MHt (m)"], errors="ignore")
print("📄 Columnas actuales:  ", list(df.columns))

# Mostrar diferencia
eliminadas = set(df_original.columns) - set(df.columns)
print("🗑️ Columnas eliminadas:", eliminadas)
df = rename_columns_using_schema(df)
print(df.head(1))


📄 Columnas originales: ['FarmerCode', 'Contract', 'Property', 'PlantingYear', 'PropertyID', 'Plot#', 'TreeID', 'TPA_1/10ac', 'TreeStatus', 'DBH (in)', 'THt (ft)', 'MHt (ft)', 'DAP (cm)', 'THt (m)', 'MHt (m)', 'BA (sqft/ac)', 'BA_ac', 'VOL_m3', 'VOL_m3ac', 'MBF', 'MBF_ac']
📄 Columnas actuales:   ['FarmerCode', 'Contract', 'Property', 'PlantingYear', 'PropertyID', 'Plot#', 'TreeID', 'TPA_1/10ac', 'TreeStatus', 'DBH (in)', 'THt (ft)', 'MHt (ft)', 'BA (sqft/ac)', 'BA_ac', 'VOL_m3', 'VOL_m3ac', 'MBF', 'MBF_ac']
🗑️ Columnas eliminadas: {'DAP (cm)', 'MHt (m)', 'THt (m)'}
   FarmerCode contractcode   farmername  PlantingYear  PropertyID  plot  \
0       10001       US0004  BruceGamble          2016         NaN     1   

   tree_number  TPA_1/10ac      Status  dbh_in  tht_ft  merch_ht_ft  \
0            1        10.0  Suppressed     2.0     NaN          NaN   

   BA (sqft/ac)    BA_ac  VOL_m3  VOL_m3ac  MBF  MBF_ac  
0      0.021816  0.21816     NaN       NaN  NaN     NaN  


In [3]:
# CALCULADORES que faltaban:
engine = get_engine()
df = calculate_dead_alive(df, engine)
df = calculate_doyle(df)

💻 Conectado a la base de datos helloworldtree
🌳 Doyle calculado


In [4]:
engine = get_engine()
df_sql, _ = prepare_df_for_sql(df)
df_sql = df_sql.where(pd.notna(df_sql), None)

# ← El segundo valor (dtypes) lo ignoramos
tabla_destino = "inventory_us_2022"

💻 Conectado a la base de datos helloworldtree


In [5]:
ensure_table(df_sql, engine, tabla_destino)

🔑 PRIMARY KEY agregada en 'id'


In [6]:
save_inventory_to_sql(df_sql, engine, tabla_destino)




=== INICIO DE IMPORTACIÓN ===
💻 Conectado a la base de datos helloworldtree
✅ Bulk insert completado: 
 'inventory_us_2022' (4999 filas)
